In [ ]:
import cvxpy as cp

# sets
nodes = ["A", "B"]
products = ["drug1"]
arcs = [("A", "B")]
T = 1   # single period toy model

N, K, A = len(nodes), len(products), len(arcs)

# params
Dist = {("A","B"): 10}
cap = 100
M = 10
storage = {"A": 200, "B": 150}
demand = {("A","drug1"): 0, ("B","drug1"): 50}
I0 = {("A","drug1"): 100, ("B","drug1"): 0}

v = 2.0
h = 0.5
c_exp = {("A","drug1"): 0, ("B","drug1"): 0}

# vars
F = { (i,j,k): cp.Variable(nonneg=True) for (i,j) in arcs for k in products }
lmbd = { (i,j): cp.Variable(integer=True) for (i,j) in arcs }
y = { (i,j): cp.Variable(boolean=True) for (i,j) in arcs }
I = { (n,k): cp.Variable(nonneg=True) for n in nodes for k in products }

constraints = []
# non-neg lambda
for (i,j) in arcs:
    constraints.append(lmbd[(i,j)] >= 0)

# balance
for n in nodes:
    for k in products:
        inflow = sum(F[(i,j,k)] for (i,j) in arcs if j==n)
        outflow = sum(F[(i,j,k)] for (i,j) in arcs if i==n)
        constraints.append(
            I[(n,k)] == I0[(n,k)] + inflow - outflow - demand[(n,k)]
        )

# truck capacity
for (i,j) in arcs:
    constraints.append(sum(F[(i,j,k)] for k in products) <= cap * lmbd[(i,j)])

# big-m!
for (i,j) in arcs:
    constraints.append(lmbd[(i,j)] <= M * y[(i,j)])

# storage
for n in nodes:
    constraints.append(sum(I[(n,k)] for k in products) <= storage[n])

# objective
obj = (v * sum(Dist[(i,j)] * lmbd[(i,j)] for (i,j) in arcs) + sum(c_exp[(n,k)]*0 + h*I[(n,k)] for n in nodes for k in products))
prob = cp.Problem(cp.Minimize(obj), constraints)
prob.solve(solver=cp.GUROBI)

print("Objective:", prob.value)
for (i,j,k), var in F.items():
    print(f"Flow {k} {i}->{j}: {var.value:.2f}")
for (i,j), var in lmbd.items():
    print(f"Trucks {i}->{j}: {var.value}")
for (i,j), var in y.items():
    print(f"Route active {i}->{j}: {var.value}")
for (n,k), var in I.items():
    print(f"Inventory {n},{k}: {var.value:.2f}")



Restricted license - for non-production use only - expires 2026-11-23
Objective: 45.0
Flow drug1 A->B: 50.00
Trucks A->B: 1.0
Route active A->B: 1.0
Inventory A,drug1: 50.00
Inventory B,drug1: 0.00
